<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/HVAC_%EC%B5%9C%EC%A0%81%ED%99%94_%EC%8B%9C%EC%8A%A4%ED%85%9C_V7_(%EB%8B%A8%EC%9C%84%EB%B3%80%EA%B2%BD%2C_%EC%A0%9C%EC%95%BD%EC%88%98%EC%A0%95%2C_%EB%AA%A8%EB%8D%B8%EC%A0%80%EC%9E%A5_%EB%A1%9C%EB%93%9C%2C_%EC%B6%9C%EB%A0%A5%EA%B0%9C%EC%84%A0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import os
import joblib # 모델 저장/로드를 위해 추가

# --- Configuration ---
DEBUG_MODE = False # Facade 내부 중간 계산 과정 상세 출력 여부
TRAIN_NEW_MODELS = True # True면 새로 학습하고 저장, False면 저장된 모델 로드 시도
MODEL_SAVE_DIR = "trained_hvac_models" # 학습된 모델 저장 경로

class EnthalpyCalculator:
    def calculate_enthalpy(self, temperature_c, relative_humidity_percent, pressure_pa=101325):
        """엔탈피를 kJ/kg 단위로 계산합니다."""
        h_j_kg = (temperature_c * 1000 + (relative_humidity_percent / 100) * 2500 * 10) * (1 + (temperature_c / 100))
        return h_j_kg / 1000.0 # J/kg -> kJ/kg

    def calculate_temp_humidity_from_enthalpy(self, enthalpy_kj_kg, initial_temp_c, pressure_pa=101325):
        """엔탈피(kJ/kg)로부터 온도와 습도를 추정합니다."""
        enthalpy_j_kg = enthalpy_kj_kg * 1000.0 # kJ/kg -> J/kg

        initial_h_j_kg_approx = self.calculate_enthalpy(initial_temp_c, 50) * 1000.0 # kJ/kg -> J/kg
        delta_h_j_kg = enthalpy_j_kg - initial_h_j_kg_approx

        cp_approx = 1025 # J/kg.K
        delta_temp_c = delta_h_j_kg / cp_approx
        new_temp_c = initial_temp_c + delta_temp_c

        # 습도 변화 로직 (단순화 유지)
        new_humidity_percent = 50 + (delta_h_j_kg) / 20000.0 * 5 # J/kg 기준 delta_h 사용
        new_humidity_percent = np.clip(new_humidity_percent, 5.0, 99.0)
        new_temp_c = np.clip(new_temp_c, -15.0, 55.0)
        return new_temp_c, new_humidity_percent

def generate_dummy_data(num_days=7, freq_minutes=10, num_oac=14, num_low_ch=8, num_high_ch=8, num_ct=9):
    num_rows = int(num_days * 24 * (60 / freq_minutes))
    data_dict = {}
    time_deltas = pd.to_timedelta(np.arange(num_rows) * freq_minutes, unit='m')
    datetimes = pd.to_datetime('2023-01-01') + time_deltas
    data_dict['datetime'] = datetimes
    time_index_for_sin = np.arange(num_rows)
    day_period_in_steps = 24 * (60 / freq_minutes); month_period_in_steps = 30 * day_period_in_steps
    outdoor_temps = 15 + 10 * np.sin(2*np.pi*time_index_for_sin/month_period_in_steps) + 5 * np.sin(2*np.pi*time_index_for_sin/day_period_in_steps) + np.random.normal(0,1,num_rows)
    outdoor_rhs = 60 + 15 * np.sin(2*np.pi*time_index_for_sin/(month_period_in_steps/2)) + 10 * np.sin(2*np.pi*time_index_for_sin/day_period_in_steps + np.pi/2) + np.random.normal(0,5,num_rows)
    outdoor_rhs = np.clip(outdoor_rhs, 20, 95)
    data_dict['outdoor_temp_c'] = outdoor_temps
    data_dict['outdoor_rh_percent'] = outdoor_rhs
    enthalpy_calc = EnthalpyCalculator()
    oac_coil_types = ['preheating', 'precooling', 'cooling', 'reheating', 'humidification']
    # 설계 엔탈피 변화량 kJ/kg 단위로 변경
    design_delta_h_oac_kj_kg = {'preheating': 15.0, 'precooling': -20.0, 'cooling': -25.0, 'reheating': 12.0, 'humidification': 8.0}
    total_precooling_load_watts_series = np.zeros(num_rows)
    total_cooling_load_watts_series = np.zeros(num_rows)
    for i in range(num_oac):
        if f'oac_{i}_토출온도_c' not in data_dict:
            data_dict[f'oac_{i}_토출온도_c'], data_dict[f'oac_{i}_토출노점온도_c'], data_dict[f'oac_{i}_토출압력_pa'] = [np.zeros(num_rows) for _ in range(3)]
        for row_idx in range(num_rows):
            oac_in_temp_curr, oac_in_rh_curr = outdoor_temps[row_idx], outdoor_rhs[row_idx]
            oac_final_out_temp, oac_final_out_rh = oac_in_temp_curr, oac_in_rh_curr
            for coil_name in oac_coil_types:
                sp_temp_key, open_rate_key = f'oac_{i}_{coil_name}_set_point_temp_c', f'oac_{i}_{coil_name}_coil_open_rate'
                outlet_temp_key, outlet_rh_key = f'oac_{i}_{coil_name}_후단온도_c', f'oac_{i}_{coil_name}_후단습도_percent'
                if row_idx == 0:
                    if sp_temp_key not in data_dict: data_dict[sp_temp_key] = np.zeros(num_rows)
                    if open_rate_key not in data_dict: data_dict[open_rate_key] = np.zeros(num_rows)
                    if coil_name != 'humidification':
                        if outlet_temp_key not in data_dict: data_dict[outlet_temp_key] = np.zeros(num_rows)
                        if outlet_rh_key not in data_dict: data_dict[outlet_rh_key] = np.zeros(num_rows)
                set_point_temp_val, base_sp_offset = 0, (i % 5 - 2) * 0.1
                if coil_name == 'preheating': set_point_temp_val = np.where(oac_in_temp_curr < 18, 20, oac_in_temp_curr) + np.random.normal(0,0.1) + base_sp_offset
                elif coil_name == 'precooling': set_point_temp_val = np.where(oac_in_temp_curr > 25, 22, oac_in_temp_curr) + np.random.normal(0,0.1) + base_sp_offset
                elif coil_name == 'cooling': set_point_temp_val = 12.0 + np.random.normal(0,0.1) + base_sp_offset
                elif coil_name == 'reheating': set_point_temp_val = 22.0 + np.random.normal(0,0.1) + base_sp_offset
                elif coil_name == 'humidification':
                    set_point_temp_val = oac_in_temp_curr + (np.random.uniform(0.5,1.5)+base_sp_offset if outdoor_temps[row_idx]<15 else base_sp_offset)
                data_dict[sp_temp_key][row_idx] = np.round(set_point_temp_val, 2)
                open_rate, inlet_h_kj = 0.0, enthalpy_calc.calculate_enthalpy(oac_in_temp_curr, oac_in_rh_curr)
                target_rh_for_sp = oac_in_rh_curr
                if coil_name == 'humidification' and set_point_temp_val > oac_in_temp_curr: target_rh_for_sp = min(oac_in_rh_curr+(set_point_temp_val-oac_in_temp_curr)*10,95)
                target_h_kj = enthalpy_calc.calculate_enthalpy(set_point_temp_val, target_rh_for_sp)
                required_delta_h_kj = target_h_kj - inlet_h_kj
                if design_delta_h_oac_kj_kg[coil_name] != 0:
                    if (design_delta_h_oac_kj_kg[coil_name] > 0 and required_delta_h_kj > 0) or \
                       (design_delta_h_oac_kj_kg[coil_name] < 0 and required_delta_h_kj < 0):
                        open_rate = abs(required_delta_h_kj / design_delta_h_oac_kj_kg[coil_name])
                open_rate = np.clip(open_rate * np.random.uniform(0.9,1.1),0,1)
                data_dict[open_rate_key][row_idx] = open_rate
                actual_delta_h_kj = design_delta_h_oac_kj_kg[coil_name] * open_rate
                outlet_h_kj = inlet_h_kj + actual_delta_h_kj
                oac_coil_out_temp, oac_coil_out_rh = enthalpy_calc.calculate_temp_humidity_from_enthalpy(outlet_h_kj, oac_in_temp_curr)
                if coil_name != 'humidification':
                    data_dict[outlet_temp_key][row_idx], data_dict[outlet_rh_key][row_idx] = oac_coil_out_temp, oac_coil_out_rh
                oac_in_temp_curr, oac_in_rh_curr = oac_coil_out_temp, oac_coil_out_rh
                if coil_name == 'humidification': oac_final_out_temp, oac_final_out_rh = oac_coil_out_temp, oac_coil_out_rh
                air_mass_flow_per_oac = 12.0
                if coil_name == 'precooling': total_precooling_load_watts_series[row_idx] += abs(actual_delta_h_kj * 1000 * air_mass_flow_per_oac) # kJ to J
                elif coil_name == 'cooling': total_cooling_load_watts_series[row_idx] += abs(actual_delta_h_kj * 1000 * air_mass_flow_per_oac) # kJ to J
            data_dict[f'oac_{i}_토출온도_c'][row_idx] = oac_final_out_temp
            data_dict[f'oac_{i}_토출노점온도_c'][row_idx] = oac_final_out_temp - (100 - oac_final_out_rh)/5
            data_dict[f'oac_{i}_토출압력_pa'][row_idx] = 101325 + np.random.uniform(100,300)
    data_dict['total_precooling_load_watts'], data_dict['total_cooling_load_watts'] = total_precooling_load_watts_series, total_cooling_load_watts_series
    for temp_type_prefix, num_chillers in [("low_temp_chiller", num_low_ch), ("high_temp_chiller", num_high_ch)]:
        load_key = 'total_cooling_load_watts' if "low_temp" in temp_type_prefix else 'total_precooling_load_watts'
        mean_load = np.mean(data_dict[load_key]); mean_load = 1 if mean_load == 0 else mean_load
        data_dict[f'{temp_type_prefix}_main_차압압력_bar'] = np.clip(1.5+np.random.normal(0,0.1,num_rows)-(data_dict[load_key]/mean_load-1)*0.2, 1.0, 2.5)
        data_dict[f'{temp_type_prefix}_main_차압개도율_percent'] = np.clip(60+np.random.normal(0,5,num_rows)+(data_dict[load_key]/mean_load-1)*10, 10, 100)
        data_dict[f'{temp_type_prefix}_sub_차압개도율_percent'] = np.clip(50+np.random.normal(0,5,num_rows)+(data_dict[load_key]/mean_load-1)*8, 10, 100)
        data_dict[f'{temp_type_prefix}_main_supply_압력_pa'] = 200000+np.random.uniform(-5000,5000,num_rows)
        data_dict[f'{temp_type_prefix}_main_supply_온도_c'] = (7.0 if "low_temp" in temp_type_prefix else 10.0)+np.random.normal(0,0.2,num_rows)
        data_dict[f'{temp_type_prefix}_main_return_압력_pa'] = 150000+np.random.uniform(-5000,5000,num_rows)
        chiller_cap_unit = (200000 if "low_temp" in temp_type_prefix else 250000)
        data_dict[f'{temp_type_prefix}_main_return_온도_c'] = np.clip(data_dict[f'{temp_type_prefix}_main_supply_온도_c']+(data_dict[load_key]/(num_chillers*chiller_cap_unit+1e-6))*5, data_dict[f'{temp_type_prefix}_main_supply_온도_c']+0.5, data_dict[f'{temp_type_prefix}_main_supply_온도_c']+10)
        active_count_series = np.clip(np.ceil(data_dict[load_key]/chiller_cap_unit).astype(int),1,num_chillers)
        data_dict[f'{temp_type_prefix}_active_count'] = active_count_series
        for j in range(num_chillers):
            is_active_roll = np.random.rand(num_rows) < (active_count_series/num_chillers)
            base_supply_temp = data_dict[f'{temp_type_prefix}_main_supply_온도_c']
            data_dict[f'{temp_type_prefix}_{j}_supply_냉수온도_c'] = np.where(is_active_roll, base_supply_temp+np.random.normal(0,0.1,num_rows), base_supply_temp+np.random.uniform(0.5,2,num_rows))
            load_per_active_chiller = data_dict[load_key]/(active_count_series+1e-6); individual_chiller_cap_delta_t = 150000
            delta_t_chilled = (load_per_active_chiller/(individual_chiller_cap_delta_t+1e-6))*5
            data_dict[f'{temp_type_prefix}_{j}_return_냉수온도_c'] = np.clip(data_dict[f'{temp_type_prefix}_{j}_supply_냉수온도_c']+np.where(is_active_roll,delta_t_chilled+np.random.normal(0,0.2,num_rows),np.random.uniform(0.1,0.5,num_rows)), data_dict[f'{temp_type_prefix}_{j}_supply_냉수온도_c']+0.2, data_dict[f'{temp_type_prefix}_{j}_supply_냉수온도_c']+12)
            data_dict[f'{temp_type_prefix}_{j}_supply_냉각수온도_c'] = outdoor_temps+5+np.random.normal(0,0.5,num_rows)
            data_dict[f'{temp_type_prefix}_{j}_return_냉각수온도_c'] = np.clip(data_dict[f'{temp_type_prefix}_{j}_supply_냉각수온도_c']+np.where(is_active_roll,3+np.random.normal(0,0.3,num_rows),np.random.uniform(0.1,0.5,num_rows)), data_dict[f'{temp_type_prefix}_{j}_supply_냉각수온도_c']+0.2, data_dict[f'{temp_type_prefix}_{j}_supply_냉각수온도_c']+10)
    for k in range(num_ct):
        data_dict[f'cooling_tower_{k}_supply_냉각수온도_c'] = outdoor_temps+3+np.random.normal(0,0.5,num_rows)
        data_dict[f'cooling_tower_{k}_return_냉각수온도_c'] = np.clip(data_dict[f'cooling_tower_{k}_supply_냉각수온도_c']+np.random.uniform(3,7,num_rows), data_dict[f'cooling_tower_{k}_supply_냉각수온도_c']+1.0, data_dict[f'cooling_tower_{k}_supply_냉각수온도_c']+15)
        data_dict[f'cooling_tower_{k}_supply_수조레벨_percent'] = np.random.uniform(70,95,num_rows)
    df = pd.DataFrame(data_dict)
    return df

def create_dummy_csv_second_data(filepath="dummy_hvac_seconds_data.csv", num_hours=1):
    num_seconds = num_hours * 3600; data = pd.DataFrame(); data['datetime'] = pd.to_datetime('2023-07-01 00:00:00')+pd.to_timedelta(np.arange(num_seconds),unit='s')
    base_outdoor_temp = 28+2*np.sin(2*np.pi*np.arange(num_seconds)/(3600*6)); data['outdoor_temp_c'] = base_outdoor_temp+np.random.normal(0,0.1,num_seconds)
    base_outdoor_rh = 70+5*np.sin(2*np.pi*np.arange(num_seconds)/(3600*4)); data['outdoor_rh_percent'] = np.clip(base_outdoor_rh+np.random.normal(0,1,num_seconds),40,90)
    for i in range(14): # 모든 OAC에 대해 기본 컬럼 생성
        for coil in ['preheating', 'precooling', 'cooling', 'reheating', 'humidification']:
            data[f'oac_{i}_{coil}_set_point_temp_c'] = 20.0 # 예시 값
            data[f'oac_{i}_{coil}_coil_open_rate'] = np.random.rand(num_seconds) * 0.1 # 예시 값
            if coil != 'humidification':
                 data[f'oac_{i}_{coil}_후단온도_c'] = data['outdoor_temp_c'] # 매우 단순화
                 data[f'oac_{i}_{coil}_후단습도_percent'] = data['outdoor_rh_percent'] # 매우 단순화
        data[f'oac_{i}_토출온도_c'] = data['outdoor_temp_c'] - 5 # 매우 단순화
    data['total_cooling_load_watts'] = 500000*np.random.rand(num_seconds); data['total_precooling_load_watts'] = 100000*np.random.rand(num_seconds)
    for chtype in ['low_temp_chiller', 'high_temp_chiller']:
        data[f'{chtype}_active_count'] = np.random.randint(1, 5, num_seconds)
        data[f'{chtype}_main_차압압력_bar'] = 1.5 + np.random.rand(num_seconds) * 0.2
        data[f'{chtype}_main_차압개도율_percent'] = 60 + np.random.rand(num_seconds) * 10
        data[f'{chtype}_sub_차압개도율_percent'] = 50 + np.random.rand(num_seconds) * 10
    data.to_csv(filepath, index=False); print(f"초당 더미 CSV 파일 생성 완료: {filepath}")

def load_and_preprocess_data(data_source='dummy', csv_filepath=None, num_days_dummy=7, freq_minutes_dummy=10):
    if data_source == 'csv' and csv_filepath and os.path.exists(csv_filepath):
        print(f"CSV 파일 로드 중: {csv_filepath}")
        try:
            df_raw = pd.read_csv(csv_filepath); print(f"CSV 파일 로드 완료. 원본 Shape: {df_raw.shape}")
            if 'datetime' not in df_raw.columns: raise ValueError("'datetime' 컬럼이 CSV 파일에 없습니다.")
            df_raw['datetime'] = pd.to_datetime(df_raw['datetime']); df_raw = df_raw.set_index('datetime')
            numeric_cols = df_raw.select_dtypes(include=np.number).columns
            df_resampled = df_raw[numeric_cols].resample(f'{freq_minutes_dummy}T').mean().reset_index()
            print(f"10분 단위 리샘플링 완료. Shape: {df_resampled.shape}")
            required_cols_check = ['outdoor_temp_c','outdoor_rh_percent','oac_0_cooling_set_point_temp_c','total_cooling_load_watts'] # 최소한의 체크
            missing_cols = [col for col in required_cols_check if col not in df_resampled.columns]
            if missing_cols:
                 print(f"경고: CSV에 필수 컬럼 부족: {missing_cols}. 더미 데이터 사용."); return generate_dummy_data(num_days_dummy,freq_minutes_dummy)
            print("CSV 데이터 전처리 완료."); return df_resampled.fillna(method='ffill').fillna(method='bfill')
        except Exception as e: print(f"CSV 처리 오류: {e}. 더미 데이터 사용."); return generate_dummy_data(num_days_dummy,freq_minutes_dummy)
    else:
        if data_source == 'csv': print(f"경고: CSV 경로 문제 ('{csv_filepath}'). 더미 데이터 사용.")
        return generate_dummy_data(num_days=num_days_dummy, freq_minutes=freq_minutes_dummy)

class OACModel:
    def __init__(self, num_oac_units_in_system=14):
        self.num_oac_units_in_system = num_oac_units_in_system
        self.enthalpy_calculator = EnthalpyCalculator()
        self.coil_order = ['preheating', 'precooling', 'cooling', 'reheating', 'humidification']
        self.trained_models = {}
        self.feature_names_per_coil = {}
        self.design_delta_h = {'preheating': 15.0, 'precooling': -20.0, 'cooling': -25.0, 'reheating': 12.0, 'humidification': 8.0} # kJ/kg 단위
    def _ensure_dir(self, directory_path):
        if not os.path.exists(directory_path): os.makedirs(directory_path)
    def save_models(self, directory_path):
        self._ensure_dir(directory_path)
        joblib.dump(self.trained_models, os.path.join(directory_path, "oac_trained_models.joblib"))
        joblib.dump(self.feature_names_per_coil, os.path.join(directory_path, "oac_feature_names.joblib"))
        print(f"OAC 모델 저장 완료: {directory_path}")
    def load_models(self, directory_path):
        try:
            self.trained_models = joblib.load(os.path.join(directory_path, "oac_trained_models.joblib"))
            self.feature_names_per_coil = joblib.load(os.path.join(directory_path, "oac_feature_names.joblib"))
            print(f"OAC 모델 로드 완료: {directory_path}")
            return True
        except FileNotFoundError: print(f"OAC 모델 파일 없음: {directory_path}. 새로 학습 필요."); return False
        except Exception as e: print(f"OAC 모델 로드 오류: {e}. 새로 학습 필요."); return False
    def train_coil_models(self, df_train_data_full):
        if 'outdoor_enthalpy' not in df_train_data_full.columns:
             df_train_data_full['outdoor_enthalpy'] = self.enthalpy_calculator.calculate_enthalpy(df_train_data_full['outdoor_temp_c'], df_train_data_full['outdoor_rh_percent'])
        for oac_idx in range(self.num_oac_units_in_system):
            self.trained_models[oac_idx] = {}
            self.feature_names_per_coil[oac_idx] = {}
            for coil_type in self.coil_order:
                target_col = f'oac_{oac_idx}_{coil_type}_coil_open_rate'
                if target_col not in df_train_data_full.columns: continue
                temp_df_train = df_train_data_full.copy()
                current_features_generic_names, current_features_specific_names = [], []
                current_features_generic_names.extend(['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy'])
                current_features_specific_names.extend(['outdoor_temp_c', 'outdoor_rh_percent', 'outdoor_enthalpy'])
                if coil_type == 'preheating':
                    current_features_generic_names.append('current_coil_set_point_temp_c')
                    current_features_specific_names.append(f'oac_{oac_idx}_{coil_type}_set_point_temp_c')
                else:
                    prev_coil = self.coil_order[self.coil_order.index(coil_type)-1]
                    if prev_coil == 'humidification':
                        prev_outlet_temp_col_specific, prev_outlet_rh_col_specific = f'oac_{oac_idx}_토출온도_c', 'outdoor_rh_percent'
                    else:
                        prev_outlet_temp_col_specific, prev_outlet_rh_col_specific = f'oac_{oac_idx}_{prev_coil}_후단온도_c', f'oac_{oac_idx}_{prev_coil}_후단습도_percent'
                    if prev_outlet_temp_col_specific not in temp_df_train.columns or prev_outlet_rh_col_specific not in temp_df_train.columns: continue
                    prev_outlet_h_col_calculated_specific = f'oac_{oac_idx}_{prev_coil}_후단엔탈피_calculated' # kJ/kg
                    temp_df_train[prev_outlet_h_col_calculated_specific] = self.enthalpy_calculator.calculate_enthalpy(temp_df_train[prev_outlet_temp_col_specific], temp_df_train[prev_outlet_rh_col_specific])
                    current_features_generic_names.extend([f'prev_coil_outlet_temp_c', f'prev_coil_outlet_rh_percent', f'prev_coil_outlet_h_actual'])
                    current_features_specific_names.extend([prev_outlet_temp_col_specific, prev_outlet_rh_col_specific, prev_outlet_h_col_calculated_specific])
                    current_features_generic_names.append('current_coil_set_point_temp_c')
                    current_features_specific_names.append(f'oac_{oac_idx}_{coil_type}_set_point_temp_c')
                self.feature_names_per_coil[oac_idx][coil_type] = current_features_generic_names
                X = temp_df_train[current_features_specific_names].fillna(method='ffill').fillna(method='bfill')
                X.columns = current_features_generic_names
                y = temp_df_train[target_col]
                if X.empty or y.empty or X.shape[0] != y.shape[0]: continue
                X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=oac_idx)
                model = RandomForestRegressor(n_estimators=10, random_state=42, n_jobs=-1, max_depth=5, min_samples_split=10, min_samples_leaf=5)
                model.fit(X_train, y_train)
                self.trained_models[oac_idx][coil_type] = model
    def predict_coil_open_rates(self, outdoor_temp_c, outdoor_rh_percent, oac_set_points_by_unit, modified_oac_coil_states_all_units=None):
        if not self.trained_models or not self.feature_names_per_coil:
            all_oac_predictions = {}
            for oac_idx in range(self.num_oac_units_in_system):
                current_oac_set_points = oac_set_points_by_unit.get(f'oac_{oac_idx}', {})
                current_oac_modified_states = modified_oac_coil_states_all_units.get(f'oac_{oac_idx}') if modified_oac_coil_states_all_units else None
                all_oac_predictions[f'oac_{oac_idx}'] = self._predict_single_oac_rule_based(outdoor_temp_c, outdoor_rh_percent, current_oac_set_points, current_oac_modified_states)
            return all_oac_predictions
        if modified_oac_coil_states_all_units is None: modified_oac_coil_states_all_units = {}
        all_oac_results = {}
        outdoor_enthalpy_val_kj = self.enthalpy_calculator.calculate_enthalpy(outdoor_temp_c, outdoor_rh_percent) # kJ/kg
        for oac_idx in range(self.num_oac_units_in_system):
            current_oac_unit_results = {}
            current_temp_c_for_sim, current_rh_percent_for_sim = outdoor_temp_c, outdoor_rh_percent
            prev_coil_actual_outlet_temp_for_pred, prev_coil_actual_outlet_rh_for_pred = outdoor_temp_c, outdoor_rh_percent
            oac_specific_modified_states = modified_oac_coil_states_all_units.get(f'oac_{oac_idx}', {})
            current_oac_unit_set_points = oac_set_points_by_unit.get(f'oac_{oac_idx}', {})
            for coil_type in self.coil_order:
                inlet_temp_c_sim, inlet_rh_percent_sim = current_temp_c_for_sim, current_rh_percent_for_sim
                inlet_h_kj_kg_sim = self.enthalpy_calculator.calculate_enthalpy(inlet_temp_c_sim, inlet_rh_percent_sim) # kJ/kg
                predicted_open_rate = 0.0
                if coil_type in oac_specific_modified_states:
                    predicted_open_rate = oac_specific_modified_states[coil_type]
                elif oac_idx in self.trained_models and coil_type in self.trained_models[oac_idx]:
                    model = self.trained_models[oac_idx][coil_type]
                    generic_feature_names = self.feature_names_per_coil.get(oac_idx, {}).get(coil_type)
                    if not generic_feature_names: predicted_open_rate = 0.0
                    else:
                        feature_values_map = {
                            'outdoor_temp_c': outdoor_temp_c, 'outdoor_rh_percent': outdoor_rh_percent, 'outdoor_enthalpy': outdoor_enthalpy_val_kj,
                            'prev_coil_outlet_temp_c': prev_coil_actual_outlet_temp_for_pred,
                            'prev_coil_outlet_rh_percent': prev_coil_actual_outlet_rh_for_pred,
                            'prev_coil_outlet_h_actual': self.enthalpy_calculator.calculate_enthalpy(prev_coil_actual_outlet_temp_for_pred, prev_coil_actual_outlet_rh_for_pred), # kJ/kg
                            'current_coil_set_point_temp_c': current_oac_unit_set_points.get(coil_type, inlet_temp_c_sim),
                        }
                        feature_values_for_pred_list = [feature_values_map[name] for name in generic_feature_names]
                        try:
                            input_df = pd.DataFrame([feature_values_for_pred_list], columns=generic_feature_names)
                            predicted_open_rate = model.predict(input_df)[0]
                            predicted_open_rate = np.clip(predicted_open_rate, 0.0, 1.0)
                        except Exception: predicted_open_rate = 0.0
                else: predicted_open_rate = 0.0
                actual_delta_h_kj_kg = self.design_delta_h[coil_type] * predicted_open_rate # kJ/kg
                outlet_h_kj_kg_sim = inlet_h_kj_kg_sim + actual_delta_h_kj_kg # kJ/kg
                current_temp_c_for_sim, current_rh_percent_for_sim = self.enthalpy_calculator.calculate_temp_humidity_from_enthalpy(outlet_h_kj_kg_sim, inlet_temp_c_sim)
                current_oac_unit_results[coil_type] = {'open_rate': predicted_open_rate, 'inlet_temp_c': inlet_temp_c_sim, 'inlet_rh_percent': inlet_rh_percent_sim, 'inlet_h_j_kg': inlet_h_kj_kg_sim, 'outlet_temp_c': current_temp_c_for_sim, 'outlet_rh_percent': current_rh_percent_for_sim, 'outlet_h_j_kg': outlet_h_kj_kg_sim, 'delta_h_j_kg': actual_delta_h_kj_kg,'target_set_point_temp_c': current_oac_unit_set_points.get(coil_type, 'N/A')}
                prev_coil_actual_outlet_temp_for_pred, prev_coil_actual_outlet_rh_for_pred = current_temp_c_for_sim, current_rh_percent_for_sim
            current_oac_unit_results['final_outlet_temp_c'], current_oac_unit_results['final_outlet_rh_percent'] = current_temp_c_for_sim, current_rh_percent_for_sim
            current_oac_unit_results['final_outlet_h_j_kg'] = self.enthalpy_calculator.calculate_enthalpy(current_temp_c_for_sim, current_rh_percent_for_sim) # kJ/kg
            all_oac_results[f'oac_{oac_idx}'] = current_oac_unit_results
        return all_oac_results
    def _predict_single_oac_rule_based(self, outdoor_temp_c, outdoor_rh_percent, oac_set_points_this_unit, modified_coil_states_this_oac=None):
        if modified_coil_states_this_oac is None: modified_coil_states_this_oac = {}
        current_temp_c, current_rh_percent = outdoor_temp_c, outdoor_rh_percent
        unit_results = {}
        for coil_type in self.coil_order:
            inlet_temp_c, inlet_rh_percent = current_temp_c, current_rh_percent
            inlet_h_kj_kg = self.enthalpy_calculator.calculate_enthalpy(inlet_temp_c, inlet_rh_percent) # kJ/kg
            target_set_point_temp_c = oac_set_points_this_unit.get(coil_type)
            predicted_open_rate, actual_delta_h_kj_kg = 0.0, 0.0
            if coil_type in modified_coil_states_this_oac: predicted_open_rate = modified_coil_states_this_oac[coil_type]
            elif target_set_point_temp_c is not None:
                target_rh_for_sp = inlet_rh_percent
                if coil_type == 'humidification' and target_set_point_temp_c > inlet_temp_c: target_rh_for_sp = min(inlet_rh_percent + (target_set_point_temp_c - inlet_temp_c) * 5, 95)
                target_outlet_h_kj_kg = self.enthalpy_calculator.calculate_enthalpy(target_set_point_temp_c, target_rh_for_sp) # kJ/kg
                required_delta_h_kj_kg = target_outlet_h_kj_kg - inlet_h_kj_kg # kJ/kg
                if self.design_delta_h[coil_type] != 0: # design_delta_h is kJ/kg
                    if (self.design_delta_h[coil_type] > 0 and required_delta_h_kj_kg > 0) or \
                       (self.design_delta_h[coil_type] < 0 and required_delta_h_kj_kg < 0):
                        predicted_open_rate = abs(required_delta_h_kj_kg / self.design_delta_h[coil_type])
                predicted_open_rate = np.clip(predicted_open_rate, 0.0, 1.0)
            actual_delta_h_kj_kg = self.design_delta_h[coil_type] * predicted_open_rate # kJ/kg
            outlet_h_kj_kg = inlet_h_kj_kg + actual_delta_h_kj_kg # kJ/kg
            current_temp_c, current_rh_percent = self.enthalpy_calculator.calculate_temp_humidity_from_enthalpy(outlet_h_kj_kg, inlet_temp_c)
            unit_results[coil_type] = {'open_rate': predicted_open_rate, 'inlet_temp_c': inlet_temp_c, 'inlet_rh_percent': inlet_rh_percent, 'inlet_h_j_kg': inlet_h_kj_kg, 'outlet_temp_c': current_temp_c, 'outlet_rh_percent': current_rh_percent, 'outlet_h_j_kg': outlet_h_kj_kg, 'delta_h_j_kg': actual_delta_h_kj_kg, 'target_set_point_temp_c': target_set_point_temp_c}
        unit_results['final_outlet_temp_c'], unit_results['final_outlet_rh_percent'] = current_temp_c, current_rh_percent
        unit_results['final_outlet_h_j_kg'] = self.enthalpy_calculator.calculate_enthalpy(current_temp_c, current_rh_percent) # kJ/kg
        return unit_results

class ChillerModel:
    def __init__(self): self.trained_models, self.feature_names_chiller = {}, {}
    def _ensure_dir(self, directory_path):
        if not os.path.exists(directory_path): os.makedirs(directory_path)
    def save_models(self, directory_path):
        self._ensure_dir(directory_path)
        joblib.dump(self.trained_models, os.path.join(directory_path, "chiller_trained_models.joblib"))
        joblib.dump(self.feature_names_chiller, os.path.join(directory_path, "chiller_feature_names.joblib"))
        print(f"Chiller 모델 저장 완료: {directory_path}")
    def load_models(self, directory_path):
        try:
            self.trained_models = joblib.load(os.path.join(directory_path, "chiller_trained_models.joblib"))
            self.feature_names_chiller = joblib.load(os.path.join(directory_path, "chiller_feature_names.joblib"))
            print(f"Chiller 모델 로드 완료: {directory_path}")
            return True
        except FileNotFoundError: print(f"Chiller 모델 파일 없음: {directory_path}. 새로 학습 필요."); return False
        except Exception as e: print(f"Chiller 모델 로드 오류: {e}. 새로 학습 필요."); return False
    def train_pressure_valve_models(self, df_train_data):
        for chiller_type in ["low_temp", "high_temp"]:
            load_col, active_count_col = ('total_cooling_load_watts' if chiller_type == "low_temp" else 'total_precooling_load_watts'), f'{chiller_type}_chiller_active_count'
            current_feature_cols = [load_col, active_count_col]; self.feature_names_chiller[chiller_type] = current_feature_cols
            target_cols = [f'{chiller_type}_chiller_main_차압압력_bar', f'{chiller_type}_chiller_main_차압개도율_percent', f'{chiller_type}_chiller_sub_차압개도율_percent']
            if not all(col in df_train_data.columns for col in current_feature_cols + target_cols): continue
            X, type_models = df_train_data[current_feature_cols].fillna(method='ffill').fillna(method='bfill'), {}
            for target_col in target_cols:
                y = df_train_data[target_col]
                if X.empty or y.empty or X.shape[0] != y.shape[0]: continue
                X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)
                model = RandomForestRegressor(n_estimators=10, random_state=42, n_jobs=-1, max_depth=5, min_samples_split=10,min_samples_leaf=5); model.fit(X_train, y_train)
                type_models[target_col.replace(f'{chiller_type}_chiller_', '')] = model
            self.trained_models[chiller_type] = type_models
    def predict_pressure_valve(self, current_total_load_watts, num_active_chillers_of_type, chiller_type="low_temp"):
        default_preds = {"main_차압압력_bar": 1.5, "main_차압개도율_percent": 60.0, "sub_차압개도율_percent": 50.0}
        if chiller_type not in self.trained_models or not self.trained_models[chiller_type] or chiller_type not in self.feature_names_chiller: return default_preds
        models_for_type, feature_cols_for_prediction = self.trained_models[chiller_type], self.feature_names_chiller[chiller_type]
        input_df_for_pred = pd.DataFrame([[current_total_load_watts, num_active_chillers_of_type]], columns=feature_cols_for_prediction)
        predictions = {}
        for output_key, model in models_for_type.items():
            try: predictions[output_key] = model.predict(input_df_for_pred)[0]
            except Exception: predictions[output_key] = default_preds.get(output_key, 0)
        for key in default_preds:
            if key not in predictions: predictions[key] = default_preds[key]
        return predictions

class ChillerOptimizer:
    def __init__(self, num_low_temp_chillers=8, num_high_temp_chillers=8):
        self.num_low_temp_chillers, self.num_high_temp_chillers = num_low_temp_chillers, num_high_temp_chillers
        self.chiller_specs = {"low_temp": [{"id": i, "max_capacity_watts": 200000, "cop_points": {0.3:3.0,0.50:3.8,0.7:3.5,1.0:3.0}} for i in range(num_low_temp_chillers)], "high_temp": [{"id": i, "max_capacity_watts": 250000, "cop_points": {0.3:3.2,0.50:4.0,0.7:3.7,1.0:3.2}} for i in range(num_high_temp_chillers)]} # target_load_percentage에 맞게 COP 포인트 조정
        for type_key in self.chiller_specs: self.chiller_specs[type_key] = sorted(self.chiller_specs[type_key], key=lambda x: self._get_cop_at_load(x['cop_points'], 0.50), reverse=True)
        self.target_load_percentage = 0.50 # 목표 부하율 50%
        self.min_load_percentage = 0.30
        self.max_load_percentage = 0.70 # 최대 부하율 70%

    def _get_cop_at_load(self, cop_points, load_percentage):
        if not cop_points: return 3.0
        load_percentage = np.clip(load_percentage, 0.0, 1.0)
        sorted_load_points = sorted(cop_points.keys())
        if not sorted_load_points: return 3.0
        if load_percentage == 0: return cop_points[sorted_load_points[0]]
        if load_percentage <= sorted_load_points[0]: return cop_points[sorted_load_points[0]]
        if load_percentage >= sorted_load_points[-1]: return cop_points[sorted_load_points[-1]]
        for i in range(len(sorted_load_points) - 1):
            x1, y1, x2, y2 = sorted_load_points[i],cop_points[sorted_load_points[i]],sorted_load_points[i+1],cop_points[sorted_load_points[i+1]]
            if x1 <= load_percentage < x2: return y1+(y2-y1)*(load_percentage-x1)/(x2-x1) if (x2-x1)!=0 else y1
        return cop_points[sorted_load_points[-1]]

    def optimize_chiller_operation(self, required_total_load_watts, chiller_type="low_temp"):
        specs_list, num_total_chillers = self.chiller_specs[chiller_type], len(self.chiller_specs[chiller_type])
        best_config = {'num_active':0,'active_chiller_ids':[],'load_percentages':[],'total_power':float('inf'),'avg_load_percentage':0,'meets_target_load_rate':False}
        if required_total_load_watts <= 1e-3: return 0, [], [], 0, 0, True
        for num_active in range(1, num_total_chillers + 1):
            current_chiller_selection_specs = specs_list[:num_active]
            total_max_cap_selected = sum(s['max_capacity_watts'] for s in current_chiller_selection_specs)
            if total_max_cap_selected*self.max_load_percentage < required_total_load_watts or \
               (total_max_cap_selected*self.min_load_percentage > required_total_load_watts and required_total_load_watts > 0 and num_active > 1): continue
            load_per_chiller_target = required_total_load_watts / num_active
            current_load_percentages, current_total_power, possible_combination, active_ids_temp = [], 0, True, []
            for chiller_spec in current_chiller_selection_specs:
                active_ids_temp.append(chiller_spec['id']); load_percentage_this_chiller = load_per_chiller_target / chiller_spec['max_capacity_watts']
                if not (self.min_load_percentage <= load_percentage_this_chiller <= self.max_load_percentage): # 최대 부하율 제약 체크
                    possible_combination = False; break
                current_load_percentages.append(load_percentage_this_chiller)
                load_on_this_chiller = chiller_spec['max_capacity_watts'] * load_percentage_this_chiller
                power_consumption_this_chiller = 0
                if load_on_this_chiller > 1e-3:
                    cop = self._get_cop_at_load(chiller_spec['cop_points'], load_percentage_this_chiller)
                    if cop <= 1e-6: possible_combination = False; break
                    power_consumption_this_chiller = load_on_this_chiller / cop
                current_total_power += power_consumption_this_chiller
            if not possible_combination: continue
            avg_load_percentage = sum(current_load_percentages) / num_active if num_active > 0 else 0
            meets_target = abs(avg_load_percentage - self.target_load_percentage) < 0.05 # 목표 부하율 50% +-5%
            is_better = (not best_config['active_chiller_ids']) or \
                        (best_config['meets_target_load_rate'] and meets_target and current_total_power < best_config['total_power']) or \
                        (meets_target and not best_config['meets_target_load_rate']) or \
                        (not best_config['meets_target_load_rate'] and not meets_target and \
                         (abs(avg_load_percentage-self.target_load_percentage) < abs(best_config['avg_load_percentage']-self.target_load_percentage) or \
                          (abs(avg_load_percentage-self.target_load_percentage) == abs(best_config['avg_load_percentage']-self.target_load_percentage) and current_total_power < best_config['total_power'])))
            if is_better: best_config.update({'num_active':num_active,'active_chiller_ids':sorted(active_ids_temp),'load_percentages':current_load_percentages,'total_power':current_total_power,'avg_load_percentage':avg_load_percentage,'meets_target_load_rate':meets_target})
        if not best_config['active_chiller_ids'] and required_total_load_watts > 1e-3 :
            num_active, active_ids_temp = num_total_chillers, [s['id'] for s in specs_list]
            load_per_chiller = required_total_load_watts / num_active if num_active > 0 else 0; current_load_percentages, current_total_power = [], 0
            for chiller_spec in specs_list:
                load_percentage = load_per_chiller / chiller_spec['max_capacity_watts'] if chiller_spec['max_capacity_watts'] > 0 else 0
                current_load_percentages.append(load_percentage)
                load_on_this_chiller_fallback = chiller_spec['max_capacity_watts'] * load_percentage
                power_this_chiller_fallback = 0
                if load_on_this_chiller_fallback > 1e-3:
                    effective_load_perc_for_cop_fallback = np.clip(load_percentage, self.min_load_percentage, self.max_load_percentage)
                    cop = self._get_cop_at_load(chiller_spec['cop_points'], effective_load_perc_for_cop_fallback)
                    if cop <= 1e-6: cop = 1.0
                    power_this_chiller_fallback = load_on_this_chiller_fallback / cop
                current_total_power += power_this_chiller_fallback
            avg_lp_fallback = sum(lp for lp in current_load_percentages if lp >=0) / num_active if num_active > 0 else 0
            best_config.update({'num_active':num_active,'active_chiller_ids':sorted(active_ids_temp),'load_percentages':current_load_percentages,'total_power':current_total_power,'avg_load_percentage':avg_lp_fallback,'meets_target_load_rate':abs(avg_lp_fallback - self.target_load_percentage) < 0.10})
        return best_config['num_active'], best_config['active_chiller_ids'], best_config['load_percentages'], best_config['total_power'], best_config['avg_load_percentage'], best_config['meets_target_load_rate']

class CoolingTowerOptimizer:
    def __init__(self, num_towers=9):
        self.tower_specs = [{"id": i, "max_capacity_watts": 300000+np.random.randint(-2,3)*10000, "fan_power_watts": 7000+np.random.randint(-5,6)*100} for i in range(num_towers)]
        self.num_towers, self.sorted_towers = len(self.tower_specs), sorted(self.tower_specs, key=lambda x: x['fan_power_watts']/x['max_capacity_watts'] if x['max_capacity_watts']>0 else float('inf'))
    def optimize_tower_operation(self, total_heat_rejection_load_watts):
        active_tower_ids, current_total_capacity, current_total_fan_power = [], 0, 0
        if total_heat_rejection_load_watts <= 0: return [], 0, 0, 0
        for tower in self.sorted_towers:
            if current_total_capacity >= total_heat_rejection_load_watts: break
            active_tower_ids.append(tower['id']); current_total_capacity += tower['max_capacity_watts']; current_total_fan_power += tower['fan_power_watts']
        return sorted(active_tower_ids), current_total_fan_power, current_total_capacity, len(active_tower_ids)

class HVACSystemFacade:
    def __init__(self, oac_model_instance, chiller_model_instance):
        self.oac_model, self.chiller_model = oac_model_instance, chiller_model_instance
        self.chiller_optimizer, self.cooling_tower_optimizer = ChillerOptimizer(), CoolingTowerOptimizer()
        self.air_mass_flow_rate_per_oac_unit_kg_s = 12.0
    def predict_and_optimize(self, outdoor_temp_c, outdoor_rh_percent, oac_set_points_by_unit, chilled_water_supply_set_temp, modified_oac_coil_states_all_units=None):
        if DEBUG_MODE: print("DEBUG [Facade]: OAC 예측 시작...")
        oac_predictions_all_units = self.oac_model.predict_coil_open_rates(outdoor_temp_c, outdoor_rh_percent, oac_set_points_by_unit, modified_oac_coil_states_all_units)
        if DEBUG_MODE: print(f"DEBUG [Facade]: OAC 예측 완료. (oac_0 최종 토출 온도: {oac_predictions_all_units.get('oac_0', {}).get('final_outlet_temp_c', 'N/A'):.1f}°C)")
        total_precooling_load_watts, total_cooling_load_watts = 0, 0
        for oac_id_key, unit_pred in oac_predictions_all_units.items():
            precool_delta_h_kj = unit_pred.get('precooling', {}).get('delta_h_j_kg', 0) # kJ/kg
            cool_delta_h_kj = unit_pred.get('cooling', {}).get('delta_h_j_kg', 0)       # kJ/kg
            total_precooling_load_watts += abs(precool_delta_h_kj * 1000 * self.air_mass_flow_rate_per_oac_unit_kg_s) # kJ to J
            total_cooling_load_watts += abs(cool_delta_h_kj * 1000 * self.air_mass_flow_rate_per_oac_unit_kg_s)       # kJ to J
        if DEBUG_MODE: print(f"DEBUG [Facade]: 계산된 총 예냉 부하: {total_precooling_load_watts:,.2f} W, 총 냉각 부하: {total_cooling_load_watts:,.2f} W")
        if DEBUG_MODE: print("DEBUG [Facade]: 저온 냉동기 최적화 시작...")
        optim_low_ch_count, active_low_ids, load_dist_low, power_low, avg_load_low, meets_target_low = self.chiller_optimizer.optimize_chiller_operation(total_cooling_load_watts, "low_temp")
        if DEBUG_MODE: print(f"DEBUG [Facade]: 저온 냉동기 최적화 결과 - 가동대수: {optim_low_ch_count}, 평균부하율: {avg_load_low:.2%}, 예상전력: {power_low:,.0f}W")
        chiller_preds_low_temp = self.chiller_model.predict_pressure_valve(total_cooling_load_watts, optim_low_ch_count, "low_temp") if optim_low_ch_count > 0 else {}
        if DEBUG_MODE and optim_low_ch_count > 0: print(f"DEBUG [Facade]: 저온 냉동기 모델 예측 (차압/밸브): {chiller_preds_low_temp}")
        if DEBUG_MODE: print("DEBUG [Facade]: 고온 냉동기 최적화 시작...")
        optim_high_ch_count, active_high_ids, load_dist_high, power_high, avg_load_high, meets_target_high = self.chiller_optimizer.optimize_chiller_operation(total_precooling_load_watts, "high_temp")
        if DEBUG_MODE: print(f"DEBUG [Facade]: 고온 냉동기 최적화 결과 - 가동대수: {optim_high_ch_count}, 평균부하율: {avg_load_high:.2%}, 예상전력: {power_high:,.0f}W")
        chiller_preds_high_temp = self.chiller_model.predict_pressure_valve(total_precooling_load_watts, optim_high_ch_count, "high_temp") if optim_high_ch_count > 0 else {}
        if DEBUG_MODE and optim_high_ch_count > 0: print(f"DEBUG [Facade]: 고온 냉동기 모델 예측 (차압/밸브): {chiller_preds_high_temp}")
        total_heat_rejection_watts = total_precooling_load_watts + total_cooling_load_watts + power_low + power_high
        if DEBUG_MODE: print(f"DEBUG [Facade]: 계산된 총 방열 부하: {total_heat_rejection_watts:,.2f} W")
        if DEBUG_MODE: print("DEBUG [Facade]: 냉각탑 최적화 시작...")
        active_tower_ids, tower_fan_power, tower_total_capacity, active_tower_count = self.cooling_tower_optimizer.optimize_tower_operation(total_heat_rejection_watts)
        if DEBUG_MODE: print(f"DEBUG [Facade]: 냉각탑 최적화 결과 - 가동대수: {active_tower_count}, 총팬전력: {tower_fan_power:,.0f}W")
        return {"oac_predictions_all_units": oac_predictions_all_units, "calculated_loads": {"total_precooling_load_watts": total_precooling_load_watts, "total_cooling_load_watts": total_cooling_load_watts},
                "low_temp_chiller_optimization": {"optimal_active_count": optim_low_ch_count, "active_chiller_ids": active_low_ids, "load_distribution_percentage_per_active_chiller": load_dist_low, "total_estimated_power_watts": power_low, "average_load_percentage_of_active_chillers": avg_load_low, "meets_target_load_rate": meets_target_low, "predictions": chiller_preds_low_temp},
                "high_temp_chiller_optimization": {"optimal_active_count": optim_high_ch_count, "active_chiller_ids": active_high_ids, "load_distribution_percentage_per_active_chiller": load_dist_high, "total_estimated_power_watts": power_high, "average_load_percentage_of_active_chillers": avg_load_high, "meets_target_load_rate": meets_target_high, "predictions": chiller_preds_high_temp},
                "cooling_tower_optimization": {"total_heat_rejection_load_watts": total_heat_rejection_watts, "active_tower_ids": active_tower_ids, "active_tower_count": active_tower_count, "total_fan_power_watts": tower_fan_power, "total_active_capacity_watts": tower_total_capacity}}

if __name__ == '__main__':
    data_source_choice = 'dummy'
    csv_file_path_input = "dummy_hvac_seconds_data.csv"
    # create_dummy_csv_second_data(filepath=csv_file_path_input, num_hours=1)
    print(f"--- 1. 데이터 로드 및 전처리 (선택: {data_source_choice}) ---")
    df_for_training = load_and_preprocess_data(data_source=data_source_choice, csv_filepath=csv_file_path_input, num_days_dummy=1, freq_minutes_dummy=10)
    if df_for_training.empty: print("오류: 학습 데이터가 비어있습니다. 프로그램을 종료합니다."); exit()

    oac_model_inst = OACModel(num_oac_units_in_system=14)
    chiller_model_inst = ChillerModel()

    if TRAIN_NEW_MODELS:
        print("\n--- 2. 새로운 모델 학습 및 저장 ---")
        oac_model_inst.train_coil_models(df_for_training.copy())
        oac_model_inst.save_models(MODEL_SAVE_DIR)
        chiller_model_inst.train_pressure_valve_models(df_for_training.copy())
        chiller_model_inst.save_models(MODEL_SAVE_DIR)
    else:
        print("\n--- 2. 저장된 모델 로드 시도 ---")
        oac_loaded = oac_model_inst.load_models(MODEL_SAVE_DIR)
        chiller_loaded = chiller_model_inst.load_models(MODEL_SAVE_DIR)
        if not (oac_loaded and chiller_loaded):
            print("모델 로드 실패. 새로운 모델을 학습합니다.")
            oac_model_inst.train_coil_models(df_for_training.copy())
            oac_model_inst.save_models(MODEL_SAVE_DIR)
            chiller_model_inst.train_pressure_valve_models(df_for_training.copy())
            chiller_model_inst.save_models(MODEL_SAVE_DIR)

    print("\n--- 학습된 모델 정보 확인 예시 ---")
    if 0 in oac_model_inst.trained_models and 'cooling' in oac_model_inst.trained_models[0]:
        oac_0_cooling_model = oac_model_inst.trained_models[0]['cooling']
        print(f"\n[OAC 0 - Cooling Coil Model]")
        print(f"  - 모델 타입: {type(oac_0_cooling_model)}")
        print(f"  - 모델 파라미터 (일부): n_estimators={oac_0_cooling_model.get_params()['n_estimators']}, max_depth={oac_0_cooling_model.get_params()['max_depth']}")
        if 0 in oac_model_inst.feature_names_per_coil and 'cooling' in oac_model_inst.feature_names_per_coil[0]:
            oac_0_cooling_features = oac_model_inst.feature_names_per_coil[0]['cooling']
            print(f"  - 학습 시 사용된 피처: {oac_0_cooling_features}")
            if hasattr(oac_0_cooling_model, 'feature_importances_'):
                importances = oac_0_cooling_model.feature_importances_
                print(f"  - 피처 중요도 (높을수록 중요):")
                for feature, importance in sorted(dict(zip(oac_0_cooling_features, importances)).items(), key=lambda item: item[1], reverse=True):
                    print(f"    - {feature}: {importance:.4f}")
    if 'low_temp' in chiller_model_inst.trained_models and 'main_차압압력_bar' in chiller_model_inst.trained_models['low_temp']:
        lt_chiller_pressure_model = chiller_model_inst.trained_models['low_temp']['main_차압압력_bar']
        print(f"\n[Low Temp Chiller - Main Pressure Bar Model]")
        print(f"  - 모델 타입: {type(lt_chiller_pressure_model)}")
        if 'low_temp' in chiller_model_inst.feature_names_chiller:
            lt_chiller_features = chiller_model_inst.feature_names_chiller['low_temp']
            print(f"  - 학습 시 사용된 피처: {lt_chiller_features}")
            if hasattr(lt_chiller_pressure_model, 'feature_importances_'):
                print(f"  - 피처 중요도: {dict(zip(lt_chiller_features, lt_chiller_pressure_model.feature_importances_))}")
    print("="*50)

    print("\n--- 3. HVAC 시스템 Facade 생성 (학습/로드된 모델 사용) ---")
    hvac_system = HVACSystemFacade(oac_model_instance=oac_model_inst, chiller_model_instance=chiller_model_inst)
    print("Facade 객체 생성 완료.")
    print("\n" + "="*20 + " 학습된 모델 기반 하절기 시나리오 실행 " + "="*20)
    outdoor_temp_summer, outdoor_humidity_summer = 32.0, 75.0
    oac_set_points_by_unit_summer = {}
    base_set_points = {'preheating': 18.0, 'precooling': 20.0, 'cooling': 12.0, 'reheating': 23.0, 'humidification': 23.5}
    for i in range(14):
        unit_sp = base_set_points.copy()
        unit_sp['cooling'] = round(base_set_points['cooling'] - (i % 3) * 0.2, 1)
        unit_sp['reheating'] = round(base_set_points['reheating'] + (i % 2) * 0.1, 1)
        unit_sp['humidification'] = round(base_set_points['humidification'] + (i % 4) * 0.1, 1)
        oac_set_points_by_unit_summer[f'oac_{i}'] = unit_sp
    chilled_water_supply_temp_set_summer = 7.0
    print(f"\n[입력 조건]")
    print(f"  외기 온도: {outdoor_temp_summer}°C, 외기 상대 습도: {outdoor_humidity_summer}%")
    print(f"  OAC 코일별 목표 설정 (예시 - oac_0): {oac_set_points_by_unit_summer['oac_0']}")
    print(f"  OAC 코일별 목표 설정 (예시 - oac_13): {oac_set_points_by_unit_summer['oac_13']}")
    print("-" * 50)
    summer_results = hvac_system.predict_and_optimize(outdoor_temp_summer, outdoor_humidity_summer, oac_set_points_by_unit_summer, chilled_water_supply_temp_set_summer)

    print("\n[OAC 유닛별 코일 상세 예측 결과 (학습된 모델 사용)]")
    oac_all_preds = summer_results["oac_predictions_all_units"]
    if oac_all_preds:
        for oac_idx_num_print in range(oac_model_inst.num_oac_units_in_system): # 모든 OAC 상세 출력
            oac_id_key_to_print = f'oac_{oac_idx_num_print}'
            if oac_id_key_to_print in oac_all_preds:
                print(f"\n  --- {oac_id_key_to_print.replace('_', ' ').capitalize()} 상세 정보 ---")
                unit_pred_details = oac_all_preds[oac_id_key_to_print]
                current_oac_sps_for_print = oac_set_points_by_unit_summer.get(oac_id_key_to_print, {})
                for coil_name in oac_model_inst.coil_order:
                    coil_data = unit_pred_details.get(coil_name, {})
                    target_sp = current_oac_sps_for_print.get(coil_name, 'N/A')
                    target_sp_str = f"{target_sp}°C" if isinstance(target_sp, (int, float)) else str(target_sp)
                    print(f"    코일: {coil_name.capitalize():<15} (목표 SP: {target_sp_str:<6}) | "
                          f"개도율: {coil_data.get('open_rate', 0.0)*100:>5.1f}% | "
                          f"입구: {coil_data.get('inlet_temp_c',0.0):>4.1f}°C, {coil_data.get('inlet_rh_percent',0.0):>4.1f}%, {coil_data.get('inlet_h_j_kg',0.0):>6.1f} kJ/kg | " # 단위 kJ/kg
                          f"출구: {coil_data.get('outlet_temp_c',0.0):>4.1f}°C, {coil_data.get('outlet_rh_percent',0.0):>4.1f}%, {coil_data.get('outlet_h_j_kg',0.0):>6.1f} kJ/kg | " # 단위 kJ/kg
                          f"Δh: {coil_data.get('delta_h_j_kg',0.0):>6.1f} kJ/kg") # 단위 kJ/kg
                print(f"    최종 토출: {unit_pred_details.get('final_outlet_temp_c'):.1f}°C, {unit_pred_details.get('final_outlet_rh_percent'):.1f}% ({unit_pred_details.get('final_outlet_h_j_kg'):.1f} kJ/kg)") # 단위 kJ/kg

        print("\n  --- OAC 유닛별 예측 개도율 요약 테이블 ---")
        header_cols = ["OAC 유닛"]
        for coil_name in oac_model_inst.coil_order: header_cols.append(f"{coil_name.capitalize()[:4]}.(%)")
        header_cols.extend(["T_out(°C)", "RH_out(%)"])
        col_widths = [len(h) for h in header_cols]
        table_data_for_width_calc = []
        for oac_id_key_table, unit_prediction_table in oac_all_preds.items():
            row_data_str = [oac_id_key_table.split('_')[-1]] # 'oac_0' -> '0'
            for coil_name_table in oac_model_inst.coil_order: row_data_str.append(f"{unit_prediction_table.get(coil_name_table, {}).get('open_rate', 0.0)*100:.1f}")
            row_data_str.extend([f"{unit_prediction_table.get('final_outlet_temp_c', float('nan')):.1f}", f"{unit_prediction_table.get('final_outlet_rh_percent', float('nan')):.1f}"])
            table_data_for_width_calc.append(row_data_str)
            for i, item in enumerate(row_data_str): col_widths[i] = max(col_widths[i], len(item))
        header_str = "| " + " | ".join([h.center(col_widths[i]) for i, h in enumerate(header_cols)]) + " |"
        separator_str = "|-" + "-|-".join(["-" * w for w in col_widths]) + "-|"
        print(header_str); print(separator_str)
        for row_items_str in table_data_for_width_calc:
            print(f"| " + " | ".join([item.rjust(col_widths[i]) for i, item in enumerate(row_items_str)]) + " |")

    print("\n[계산된 총 시스템 부하 (OAC 14대 기준)]")
    loads = summer_results["calculated_loads"]
    print(f"  총 예냉 코일 부하 (고온 냉동기 담당): {loads['total_precooling_load_watts']:,.0f} Watts")
    print(f"  총 주 냉각 코일 부하 (저온 냉동기 담당): {loads['total_cooling_load_watts']:,.0f} Watts")
    print("\n[저온 냉동기 시스템 최적화 및 예측 - 학습된 모델 사용]")
    low_temp_ch_opt = summer_results["low_temp_chiller_optimization"]
    print(f"  최적 가동 대수: {low_temp_ch_opt['optimal_active_count']} 대 (IDs: {low_temp_ch_opt['active_chiller_ids']}) | 평균 부하율: {low_temp_ch_opt['average_load_percentage_of_active_chillers']:.2%}")
    print(f"  예상 총 소비전력: {low_temp_ch_opt.get('total_estimated_power_watts',0):,.0f} Watts")
    print(f"  예측된 시스템 차압/밸브: {low_temp_ch_opt['predictions']}")
    print("\n[고온 냉동기 시스템 최적화 및 예측 - 학습된 모델 사용]")
    high_temp_ch_opt = summer_results["high_temp_chiller_optimization"]
    print(f"  최적 가동 대수: {high_temp_ch_opt['optimal_active_count']} 대 (IDs: {high_temp_ch_opt['active_chiller_ids']}) | 평균 부하율: {high_temp_ch_opt['average_load_percentage_of_active_chillers']:.2%}")
    print(f"  예상 총 소비전력: {high_temp_ch_opt.get('total_estimated_power_watts',0):,.0f} Watts")
    print(f"  예측된 시스템 차압/밸브: {high_temp_ch_opt['predictions']}")
    print("\n[냉각탑 최적화 결과]")
    ct_opt = summer_results["cooling_tower_optimization"]
    print(f"  총 방열 부하: {ct_opt['total_heat_rejection_load_watts']:,.0f} Watts | 최적 가동 대수: {ct_opt['active_tower_count']} 대 (IDs: {ct_opt['active_tower_ids']})")
    print("="*50)